[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/obss/sahi/blob/main/demo/inference_for_mmdet.ipynb)

## 0. Preperation

- Install latest version of SAHI and MMDetection:

In [ ]:
%pip install Pillow==9.0.0
%pip install -U sahi mmcv==1.6.0

In [ ]:
import os
os.getcwd()

- Import required modules:

In [5]:
# arrange an instance segmentation model for test
from sahi.utils.mmdet import (
    download_mmdet_cascade_mask_rcnn_model,
    download_mmdet_config,
)

# import required functions, classes
from sahi.model import MmdetDetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict, predict_fiftyone
from IPython.display import Image

- Download a cascade mask rcnn model and two test images:

In [6]:
# download cascade mask rcnn model&config
model_path = '../latest.pth'
download_mmdet_cascade_mask_rcnn_model(model_path)
# config_path = download_mmdet_config(model_name="cascade_rcnn", config_file_name="cascade_mask_rcnn_r50_fpn_1x_coco.py",)
config_path = '../finetune.py'

# download test images into demo_data folder
download_from_url('https://raw.githubusercontent.com/obss/sahi/main/demo/demo_data/small-vehicles1.jpeg', 'demo_data/small-vehicles1.jpeg')
download_from_url('https://raw.githubusercontent.com/obss/sahi/main/demo/demo_data/terrain2.png', 'demo_data/terrain2.png')

## 1. Standard Inference with a MMDetection Model

- Instantiate a detection model by defining model weight path, confing path and other parameters:

In [7]:
detection_model = MmdetDetectionModel(
    model_path=model_path,
    config_path=config_path,
    confidence_threshold=0.4,
    device="cuda:0", # or 'cuda:0'
)

load checkpoint from local path: ../latest.pth


- Perform prediction by feeding the get_prediction function with an image path and a DetectionModel instance:

In [ ]:
result = get_prediction("demo_data/small-vehicles1.jpeg", detection_model)

- Or perform prediction by feeding the get_prediction function with a numpy image and a DetectionModel instance:

In [ ]:
result = get_prediction(read_image("demo_data/small-vehicles1.jpeg"), detection_model)

- Visualize predicted bounding boxes and masks over the original image:

In [ ]:
result.export_visuals(export_dir="demo_data/")

Image("demo_data/prediction_visual.png")

## 2. Sliced Inference with a MMDetection Model

- To perform sliced prediction we need to specify slice parameters. In this example we will perform prediction over slices of 256x256 with an overlap ratio of 0.2:

In [ ]:
result = get_sliced_prediction(
    "demo_data/small-vehicles1.jpeg",
    detection_model,
    slice_height = 256,
    slice_width = 256,
    overlap_height_ratio = 0.2,
    overlap_width_ratio = 0.2
)

- Visualize predicted bounding boxes and masks over the original image:

In [ ]:
result.export_visuals(export_dir="demo_data/")

Image("demo_data/prediction_visual.png")

## 3. Prediction Result

- Predictions are returned as [sahi.prediction.PredictionResult](sahi/prediction.py), you can access the object prediction list as:

In [ ]:
object_prediction_list = result.object_prediction_list

In [ ]:
object_prediction_list[0]

- ObjectPrediction's can be converted to [COCO annotation](https://cocodataset.org/#format-data) format:

In [ ]:
result.to_coco_annotations()[0]

- ObjectPrediction's can be converted to [COCO prediction](https://github.com/i008/COCO-dataset-explorer) format:

In [ ]:
result.to_coco_predictions(image_id=1)[0]

- ObjectPrediction's can be converted to [imantics](https://github.com/jsbroks/imantics) annotation format:

In [ ]:
%pip install -U imantics

In [ ]:
result.to_imantics_annotations()[0]

- ObjectPrediction's can be converted to [fiftyone](https://github.com/voxel51/fiftyone) detection format:

In [ ]:
%pip install -U fiftyone[desktop]

In [ ]:
%pip install ipywidgets>=7.5

In [ ]:
result.to_fiftyone_detections()[0]

In [ ]:
# import fiftyone as fo
# dataset = fo.Dataset()
# dataset.add_sample(fo.Sample(filepath='demo_data/small-vehicles1.jpeg'))


In [ ]:
# with fo.ProgressBar() as pb:
#   for sample in pb(dataset):
#     detections = []
#     for d in result.to_fiftyone_detections():
#       detections.append(d)
#     sample['ssdlite-mobilenetv2'] = fo.Detections(detections=detections)
#     sample.save()

# session = fo.launch_app(dataset)

## 4. Batch Prediction

- Set model and directory parameters:

In [8]:
model_type = "mmdet"
model_path = model_path
model_config_path = config_path
model_device = "cuda:0" # or 'cuda:0'
model_confidence_threshold = 0.4

slice_height = 256
slice_width = 256
overlap_height_ratio = 0.2
overlap_width_ratio = 0.2

source_image_dir = "./demo_data/"

- Perform sliced inference on given folder:

In [ ]:
# predict(
#     model_type=model_type,
#     model_path=model_path,
#     model_config_path=config_path,
#     model_device=model_device,
#     model_confidence_threshold=model_confidence_threshold,
#     source=source_image_dir,
#     no_sliced_prediction=True
#     # slice_height=slice_height,
#     # slice_width=slice_width,
#     # overlap_height_ratio=overlap_height_ratio,
#     # overlap_width_ratio=overlap_width_ratio,
# )

In [10]:
import fiftyone as fo

In [ ]:
dataset = fo.Dataset()
for i in os.listdir('./demo_data'):
    dataset.add_sample(fo.Sample(filepath='demo_data/' + i))

In [11]:
with fo.ProgressBar() as pb:
        for sample in pb(dataset):
            prediction_result = get_prediction(
                    image=sample.filepath,
                    detection_model=detection_model,
                    shift_amount=[0, 0],
                    full_shape=None,
                    postprocess=None,
                    verbose=0,
                )
            sample[model_type] = fo.Detections(detections=prediction_result.to_fiftyone_detections())
            sample.save()

   1% |/----------------|   2/191 [147.3ms elapsed, 13.9s remaining, 13.6 samples/s] 

/home/anastasispk/Dev/mmdetection/mmdet/datasets/utils.py:66: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(


 100% |█████████████████| 191/191 [11.5s elapsed, 0s remaining, 17.0 samples/s]      


In [12]:
session = fo.launch_app()
session.dataset = dataset

In [3]:
# import fiftyone.zoo as foz

# # To download the COCO dataset for only the "person" and "car" classes
# dataset = foz.load_zoo_dataset(
#     "coco-2017",
#     split="validation",
#     label_types=["detections", "segmentations"],
#     classes=["traffic light"],
# )

 100% |██████|    1.9Gb/1.9Gb [50.3s elapsed, 0s remaining, 37.1Mb/s]      
Extracting annotations to '/home/anastasispk/fiftyone/coco-2017/raw/instances_val2017.json'
 100% |██████████████████| 191/191 [47.8s elapsed, 0s remaining, 4.4 images/s]    
Writing annotations for 191 downloaded samples to '/home/anastasispk/fiftyone/coco-2017/validation/labels.json'
Dataset info written to '/home/anastasispk/fiftyone/coco-2017/info.json'
Loading 'coco-2017' split 'validation'
 100% |█████████████████| 191/191 [3.9s elapsed, 0s remaining, 53.3 samples/s]      
Dataset 'coco-2017-validation' created
